In [3]:
%load_ext autoreload
%autoreload 2

import os
import socket

cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
print("Training results should now be reproducible.")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Host: deeplearning-vm
You are here: /home/sebelino7_gmail_com/pycharm_depl
Training results should now be reproducible.


# Train model

In [26]:

from run import run_multiple
%reload_ext autoreload
%autoreload 2

from typing import Literal
import augmentation
from augmentation import AugmentationParams
from training import TrainParams, NagParams
from freezing import MaskedFineTuningParams
from datasets import DatasetParams
from evaluation import override_param_sets, evaluate_runs_print, evaluate_runs_ci, evaluate_elapsed_time

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    class_fractions=(1.0,) * 37,
    validation_set_fraction=0.1,
)

architecture: Literal["resnet18", "resnet34", "resnet50"] = "resnet50"
baseline_params = TrainParams(
    seed=determinism.seed,
    architecture=architecture,
    n_epochs=10,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 9, 16, 23, 30),
    augmentation=AugmentationParams(
        enabled=True,
        transform=augmentation.to_transform(architecture, ["resize", "flip"]),
        dropout_rate=0.3,
    ),
    mft=MaskedFineTuningParams(
        enabled=False,
        k=100,
        impl="ours",
    ),
    validation_freq=1,
    use_scheduler=True,
    scheduler_type="plateau",
    time_limit_seconds=None,
    val_acc_target=None,
)

overrides = {
    "Full unfreeze": dict(freeze_layers=False, unfreezing_epochs=None, unfreeze_last_l_blocks=None),
    "Gradual unfreeze": dict(mft=dict(enabled=False)),
    "GPS K=10": dict(mft=dict(enabled=True, k=10)),
    "GPS K=100": dict(mft=dict(enabled=True, k=100)),
    "GPS K=1000": dict(mft=dict(enabled=True, k=1000)),
    "GPS K=4607": dict(mft=dict(enabled=True, k=4607)),
}
training_param_sets = override_param_sets(baseline_params, overrides)
results = run_multiple(dataset_params, training_param_sets, determinism, invalidate=False, trials=3)
print(results)
for label, result in results.items():
    r = list(result.values())[0]
    frac = r.mft_selected_param_count / r.mft_total_param_count
    print(f"{label}: MFT param count: {r.mft_selected_param_count}/{r.mft_total_param_count} = {100 * frac:.2f} %")
evaluate_runs_print(results)
evaluate_runs_ci(results)
evaluate_elapsed_time(results, "Full unfreeze")



Invalidate cache entry: successful
Running trial 1/3 for Full unfreeze


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Update step:   0%|          | 0/1040 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.4959, Train Acc: 57.04%, Val Acc: 91.78%
Epoch [2/10], Loss: 0.6530, Train Acc: 87.72%, Val Acc: 93.42%
Epoch [3/10], Loss: 0.3405, Train Acc: 91.34%, Val Acc: 95.07%
Epoch [4/10], Loss: 0.2457, Train Acc: 93.39%, Val Acc: 95.07%
Epoch [5/10], Loss: 0.1797, Train Acc: 95.38%, Val Acc: 95.34%
Epoch [6/10], Loss: 0.1405, Train Acc: 96.41%, Val Acc: 96.16%
Epoch [7/10], Loss: 0.1153, Train Acc: 97.10%, Val Acc: 95.89%
Epoch [8/10], Loss: 0.0839, Train Acc: 98.22%, Val Acc: 95.89%
Epoch [9/10], Loss: 0.0755, Train Acc: 98.40%, Val Acc: 95.62%
Epoch [10/10], Loss: 0.0647, Train Acc: 98.67%, Val Acc: 96.44%
Total elapsed: 180.70s, average per update step: 0.16s
[Trainer] Saved checkpoint to runs/checkpoints/917e6f1f0d1bfe29e89e29eeb17e5cfa.pth
Invalidate cache entry: successful
Running trial 2/3 for Full unfreeze


Update step:   0%|          | 0/1040 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.4906, Train Acc: 55.51%, Val Acc: 91.23%
Epoch [2/10], Loss: 0.6605, Train Acc: 86.94%, Val Acc: 92.60%
Epoch [3/10], Loss: 0.3495, Train Acc: 91.52%, Val Acc: 94.79%
Epoch [4/10], Loss: 0.2510, Train Acc: 93.51%, Val Acc: 95.07%
Epoch [5/10], Loss: 0.1807, Train Acc: 95.14%, Val Acc: 95.34%
Epoch [6/10], Loss: 0.1392, Train Acc: 96.32%, Val Acc: 95.89%
Epoch [7/10], Loss: 0.1234, Train Acc: 96.89%, Val Acc: 95.89%
Epoch [8/10], Loss: 0.0847, Train Acc: 98.22%, Val Acc: 95.62%
Epoch [9/10], Loss: 0.0762, Train Acc: 98.37%, Val Acc: 95.34%
Epoch [10/10], Loss: 0.0604, Train Acc: 98.85%, Val Acc: 95.62%
Total elapsed: 180.58s, average per update step: 0.16s
[Trainer] Saved checkpoint to runs/checkpoints/f680d3df1a66e51045c5e96ec6f37a67.pth
Invalidate cache entry: successful
Running trial 3/3 for Full unfreeze


Update step:   0%|          | 0/1040 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.4612, Train Acc: 58.16%, Val Acc: 91.78%
Epoch [2/10], Loss: 0.6464, Train Acc: 87.51%, Val Acc: 93.97%
Epoch [3/10], Loss: 0.3490, Train Acc: 91.10%, Val Acc: 95.07%


Exception ignored in: <function tqdm.__del__ at 0x7f8132f22680>
Traceback (most recent call last):
  File "/opt/python/3.10/lib/python3.10/site-packages/tqdm/std.py", line 1147, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

# Test model

In [10]:

from evaluation import evaluate_final_test_accuracy

labelled_data_fractions = [1.0, 0.5]
ssl_params = dict(fixmatch=True, labelled_data_fraction=0.5, pseudo_threshold=0.95, unsup_weight=0.5)

eval_training_params = baseline_params.copy()
eval_training_params.fixmatch = True
eval_training_params.pseudo_threshold = 0.95
eval_training_params.unsup_weight = 0.5

overrides = {
    "No GPS": dict(mft=dict(enabled=False)),
    "GPS K=10": dict(mft=dict(enabled=True, k=10)),
    "GPS K=100": dict(mft=dict(enabled=True, k=100)),
    "GPS K=1000": dict(mft=dict(enabled=True, k=1000)),
    "GPS K=4607": dict(mft=dict(enabled=True, k=4607)),
}
training_param_sets = override_param_sets(eval_training_params, overrides)

for fraction in labelled_data_fractions:
    eval_dataset_params = dataset_params.copy()
    eval_dataset_params.labelled_data_fraction = fraction
    for label, training_param_set in training_param_sets.items():
        if fraction != 1.0:
            label = f"SSL {label}"
        print(f"Evaluating '{label}'")
        evaluate_final_test_accuracy(
            eval_dataset_params,
            training_param_set,
            determinism,
            trials=3,
            display_misclassified=False,
            invalidate=False,
        )

Evaluating 'No GPS'
Test size: 3669
Invalidate cache entry: successful
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...


Update step:   0%|          | 0/1150 [00:00<?, ?it/s]

Epoch [3/10], Loss: 0.4681, Train Acc: 89.51%
Epoch [4/10], Loss: 0.3263, Train Acc: 91.68%
Epoch [5/10], Loss: 0.2459, Train Acc: 93.61%
Epoch [6/10], Loss: 0.2041, Train Acc: 94.54%
Epoch [7/10], Loss: 0.1628, Train Acc: 95.35%
Epoch [8/10], Loss: 0.1453, Train Acc: 96.22%
Epoch [9/10], Loss: 0.1267, Train Acc: 96.63%
Epoch [10/10], Loss: 0.0966, Train Acc: 97.42%
Total elapsed: 109.29s, average per update step: 0.10s


...

/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...


Update step:   0%|          | 0/1150 [00:00<?, ?it/s]

Epoch [1/10], Loss: 3.4644, Train Acc: 16.09%
Epoch [2/10], Loss: 3.1722, Train Acc: 31.98%
Epoch [3/10], Loss: 2.9347, Train Acc: 35.90%
Epoch [4/10], Loss: 2.7589, Train Acc: 38.40%
Epoch [5/10], Loss: 2.6263, Train Acc: 39.32%
Epoch [6/10], Loss: 2.5318, Train Acc: 39.21%
Epoch [7/10], Loss: 2.4336, Train Acc: 39.48%
Epoch [8/10], Loss: 2.3859, Train Acc: 40.41%
Epoch [9/10], Loss: 2.3204, Train Acc: 41.30%


KeyboardInterrupt: 